<a href="https://colab.research.google.com/github/Arwah45/Personal_Projects/blob/main/air_pollution_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
temp_df = pd.read_csv("temperature.csv")
humidity_df = pd.read_csv("humidity.csv")
co_df = pd.read_csv("CO.csv")
alcohol_df = pd.read_csv("alcohol.csv")
smoke_df = pd.read_csv("smoke.csv")

In [ ]:
# Filter data based on feed_id
temperature_df = merged_df[merged_df['feed_id'] == 2783700]
humidity_df = merged_df[merged_df['feed_id'] == 2783702]
co_df = merged_df[merged_df['feed_id'] == 2783705]
alcohol_df = merged_df[merged_df['feed_id'] == 2783703]
smoke_df = merged_df[merged_df['feed_id'] == 2783704]

In [ ]:
merged_df = (
    temperature_df.merge(humidity_df, on="created_at", suffixes=("_temp", "_humidity"))
    .merge(co_df, on="created_at", suffixes=("_humidity_co", "_co"))
    .merge(alcohol_df, on="created_at", suffixes=("_co_alcohol", "_alcohol"))
    .merge(smoke_df, on="created_at", suffixes=("_alcohol_smoke", "_smoke"))
)

In [ ]:
# Create a sample DataFrame
data = {
    "created_at": ["2023-04-01", "2023-04-02", "2023-04-03"],
    "id_temp": [1, 2, 3],
    "value_temp": [20, 22, 24],
    "id_humidity": [4, 5, 6],
    "value_humidity": [50, 60, 70],
}
merged_df = pd.DataFrame(data)


In [ ]:
# Drop the specified columns
columns_to_drop = [
    "created_at",
    "id_temp",
    "id_humidity",
]


In [ ]:
X = merged_df.drop(columns=columns_to_drop)


In [ ]:
print(merged_df.columns)

Index(['created_at', 'id_temp', 'value_temp', 'id_humidity', 'value_humidity'], dtype='object')


In [ ]:
# Define target variable
y =[["value_temp", "value_humidity"]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Predict on test set
y_pred = model.predict(X_test)


In [ ]:
# Evaluate model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 1.0097419586828951e-28


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest

In [ ]:
# Anomaly Detection using Z-score
merged_df['zscore_temp'] = zscore(merged_df['value_temp'])
merged_df['zscore_humidity'] = zscore(merged_df['value_humidity'])
anomaly_threshold = 3  # Adjust the threshold as needed
anomalies = merged_df[(abs(merged_df['zscore_temp']) > anomaly_threshold) | (abs(merged_df['zscore_humidity']) > anomaly_threshold)]


In [ ]:
# Anomaly Detection using Isolation Forest
isolation_forest = IsolationForest(contamination=0.05)
isolation_forest.fit(X_train) # Adjust the contamination parameter as needed
isolation_forest.fit(merged_df[['value_temp', 'value_humidity']])
merged_df['anomaly'] = isolation_forest.predict(merged_df[['value_temp', 'value_humidity']])
anomalies_isolation_forest = merged_df[merged_df['anomaly'] == -1]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [ ]:
anomalies = isolation_forest.predict(X_train)

In [ ]:
# Print anomalies
print("Anomalies:")
print(anomalies)

Anomalies:
[1 1]


In [ ]:
print("Anomalies (Z-score method):")
print(anomalies)
print("\nAnomalies (Isolation Forest method):")
print(anomalies_isolation_forest)
# print("\nCyclone Probability:", cyclone_probability)

Anomalies (Z-score method):
[1 1]

Anomalies (Isolation Forest method):
   created_at  id_temp  value_temp  id_humidity  value_humidity  zscore_temp  \
0  2023-04-01        1          20            4              50    -1.224745   

   zscore_humidity  anomaly  
0        -1.224745       -1  
